### Installing lxml to read html from Wikipedia Page

In [1]:
pip install lxml

     |████████████████████████████████| 5.8MB 6.2MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


### Installing Geopy and Folium Packages

In [1]:
!conda install -c conda-forge geopy --yes #Installing Geopy
!conda install -c conda-forge folium=0.5.0 --yes #Installing Folium

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forge
    lib

### Installing several necessary libraries

In [2]:
import pandas as pd #Importing pandas library
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Extract the tables from the Wiki Page into Dataframe and view the list of tables obtained

In [3]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') #Reading all tables in html into dataframe
for df in dfs: #Displaying all dataframes in the wiki page
    print(df)

    Postcode           Borough          Neighbourhood
0        M1A      Not assigned           Not assigned
1        M2A      Not assigned           Not assigned
2        M3A        North York              Parkwoods
3        M4A        North York       Victoria Village
4        M5A  Downtown Toronto           Harbourfront
..       ...               ...                    ...
282      M8Z         Etobicoke              Mimico NW
283      M8Z         Etobicoke     The Queensway West
284      M8Z         Etobicoke  Royal York South West
285      M8Z         Etobicoke         South of Bloor
286      M9Z      Not assigned           Not assigned

[287 rows x 3 columns]
                                                  0   \
0                                                NaN   
1  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...   
2                                                 NL   
3                                                  A   

                                               

### Store the table required into another dataframe

In [4]:
pc = dfs[0] #Selecting the required dataframe from all the dataframes available
pc

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


### Drop all row entries where Borough is not assigned

In [5]:
pc.drop(pc[pc['Borough']=='Not assigned'].index, inplace = True) #drop rows where Borough is "Not Assigned"
pc

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


### Group all rows entries with same Postcode and combine the respective neighbourhoods in those rows separated by commas 

In [6]:
pc = pc.groupby('Postcode').agg(lambda x:", ".join(set(x))) # Group values by Postcode and join neighbourhood values using "," where Post Code is same
pc

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Morningside, Guildwood, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
...,...,...
M9N,York,Weston
M9P,Etobicoke,Westmount
M9R,Etobicoke,"Richview Gardens, Kingsview Village, St. Phill..."


### Replace the Neighbourhood name with Borough Name where Neighbourhood name is not attached

In [7]:
pc.loc[pc['Neighbourhood']=='Not assigned',"Neighbourhood"]=pc.loc[pc['Neighbourhood']=='Not assigned',"Borough"] # Replacing corresponding Borough value where Neighbourhood is "Not assigned" 
pc

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Morningside, Guildwood, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
...,...,...
M9N,York,Weston
M9P,Etobicoke,Westmount
M9R,Etobicoke,"Richview Gardens, Kingsview Village, St. Phill..."


### Find the number of rows in the dataframe

In [8]:
pc.shape[0] #Number of rows

103